In [4]:
!pip install nltk
!pip install numpy
!pip install pandas

    100% |████████████████████████████████| 1.2MB 561kB/s ta 0:00:011
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/18/9c/1f/276bc3f421614062468cb1c9d695e6086d0c73d67ea363c501
Successfully built nltk


In [13]:
import nltk as nltk
import numpy as np
import pandas as pd
import collections as coll

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

def string_to_ngrams(s, n):
    # text = str(s).decode('utf-8').lower()
    text = str(s).lower()
    text = text.replace(' ', '')
    ngrams = nltk.ngrams([c for c in text], n)
    return [''.join(g) for g in ngrams]

class NaiveBayesClassifier(BaseEstimator, ClassifierMixin):  
    def __init__(self,
                 n_gram=1,
                 multimap=True,
                 count_threshold=0,
                 use_uniform_prior=False,
                 laplace_smoothing=1.):
        self.n_gram = n_gram
        self.multimap = multimap
        self.count_threshold = count_threshold
        self.use_uniform_prior = use_uniform_prior
        self.laplace_smoothing = laplace_smoothing

    def fit(self, X=None, y=None):
        W = pd.DataFrame({'X': [X], 'y': [y]})
        
        # Count features
        counts = {}
        for (xx, yy) in zip(X, y):
            category = np.zeros(5)
            category[yy] = 1
            ngrams = string_to_ngrams(xx, self.n_gram)
            if not self.multimap:
                ngrams = list(set(ngrams))
            for ngram in ngrams:
                if ngram in counts:
                    counts[ngram] = counts[ngram] + category
                else:
                    counts[ngram] = category
        counts = pd.DataFrame(counts).transpose()

        # Filter low counts
        keep = counts.apply(lambda row: sum(row) >= self.count_threshold, axis = 1)
        counts = counts[keep == 1]
        
        # Apply Laplace smoothing by adding a letter
        counts[counts.columns[-5:]] += self.laplace_smoothing
        
        # Count the # of n-grams observed in each language.
        class_counts = np.array(counts[counts.columns[-5:]].apply(lambda x: np.sum(x) * 1., axis = 0).values)
        
        # Define P(Y = y) as the proportion of n-grams observed in each language.
        if self.use_uniform_prior:
            class_priors = np.full(5, .1/5)
        else:
            class_priors = class_counts / np.sum(class_counts)
            
        # self.likelihood_ = counts.div(class_counts)
        self.likelihood_ = np.log(counts.div(class_counts)) ###
        self.counts_ = counts
        self.class_priors_ = class_priors
        
        return self

    def predict(self, X):
        predictions = []

        for obs in X:
            levels = string_to_ngrams(obs, n = self.n_gram)
            
            # joint_likelihood = np.full(5, 1.0)
            joint_likelihood = np.full(5, 0.)
            
            # Calculate joint probability
            for level in levels:
                if not level in self.likelihood_.index:
                    continue
                    
                # Calculate likelihood for X
                likelihood = self.likelihood_[self.likelihood_.index == level][:1].reset_index().values
                likelihood = np.array(np.delete(likelihood, 0).astype(float))
                
                # joint_likelihood = np.multiply(joint_likelihood, likelihood)
                joint_likelihood = joint_likelihood + likelihood
                
            # Calculate joint likelihood * class prior
            prop_posterior = np.multiply(joint_likelihood, self.class_priors_)
            
            # Calculate posterior probability
            posterior = prop_posterior / np.sum(prop_posterior)
            prediction = np.argmax(posterior)
            
            predictions = predictions + [prediction]
        
        return predictions
    
    
    def classify(self, inputs):
        return
 

In [10]:
# from sklearn.pipeline import Pipeline, FeatureUnion

X_train = pd.read_csv("data/train_set_x.csv")['Text'].values
Y_train = pd.read_csv("data/train_set_y.csv")['Category'].values
X_test  = pd.read_csv("data/test_set_x.csv")['Text'].values
nbayes = NaiveBayesClassifier()
nbayes.set_params(n_gram=1,
                  multimap=True,
                  count_threshold=25,
                  use_uniform_prior=False,
                  laplace_smoothing=10.0)
nbayes.fit(X_train, Y_train)
nbayes.likelihood_


,0,1,2,3,4
0,0.001344,0.001753,0.001752,0.001522,0.000093
1,0.001772,0.002259,0.001526,0.001774,0.000084
2,0.001578,0.001910,0.001180,0.001220,0.000074
3,0.001046,0.001202,0.000818,0.000781,0.000061
4,0.000723,0.000963,0.000624,0.000420,0.000039
5,0.000850,0.001399,0.000721,0.000559,0.000063
6,0.000602,0.001318,0.000605,0.000625,0.000039
7,0.000550,0.001704,0.000566,0.000431,0.000035
8,0.000514,0.000851,0.000534,0.000290,0.000032
9,0.000363,0.001270,0.000595,0.000377,0.000030


In [11]:
len(nbayes.likelihood_)

157

In [15]:
true_x = X_train[200000:201000]
true_y = Y_train[200000:201000]
pred_y = nbayes.predict(true_x)

loss = np.full((5,5), 1.0)
for i in range(len(pred_y)):
    loss[pred_y[i], true_y[i]] += 1
    
TPR = loss.trace() / loss.sum()
FNR = 1 - TPR

print(loss)
print(TPR)
print(FNR)

[[  44.    6.    5.    1.    3.]
 [   2.  473.   46.   12.    6.]
 [   5.   28.  196.    4.    2.]
 [   1.   10.   12.  114.    2.]
 [   1.    1.    1.    2.   48.]]
0.853658536585
0.146341463415


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:115: RuntimeWarning: invalid value encountered in divide


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score

import random

xx = X_train[:500]
yy = Y_train[:500]


TPR = loss.trace() / loss.sum()
FNR = 1 - TPR
print(TPR)
print(FNR)

scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro'}

print("cv_n_gram")
cv_n_gram = None
for i in range(1, 3):
    print(i)
    a = cross_validate(NaiveBayesClassifier(n_gram=i),
                       xx,
                       yy,
                       fit_params={},
                       scoring=scoring,
                       cv=2,
                       return_train_score=True)
    a_mean = pd.DataFrame(a).mean(axis=0)
    cv_n_gram = pd.concat((cv_n_gram,
                           a_mean.rename("n_gram={}".format(i))), axis=1)

print("cv_laplace_smoothing")
cv_laplace_smoothing = None
for i in [1., 10., 50.]:
    print(i)
    a = cross_validate(NaiveBayesClassifier(laplace_smoothing=i),
                       xx,
                       yy,
                       fit_params={},
                       scoring=scoring,
                       cv=2,
                       return_train_score=True)
    a_mean = pd.DataFrame(a).mean(axis=0)
    cv_laplace_smoothing = pd.concat((cv_laplace_smoothing,
                                      a_mean.rename("laplace_smoothing={}".format(i))), axis=1)
    
print("cv_count_threshold")
cv_count_threshold = None
for i in [0., 10., 50.]:
    print(i)
    a = cross_validate(NaiveBayesClassifier(count_threshold=i),
                       xx,
                       yy,
                       fit_params={},
                       scoring=scoring,
                       cv=2,
                       return_train_score=True)
    a_mean = pd.DataFrame(a).mean(axis=0)
    cv_count_threshold = pd.concat((cv_count_threshold,
                                    a_mean.rename("count_threshold={}".format(i))), axis=1)

In [19]:
print(cv_n_gram)
print(cv_laplace_smoothing)
print(cv_count_threshold)

                   n_gram=1   n_gram=2
fit_time           0.028475   0.060318
score_time        13.497908  10.938805
test_acc           0.689676   0.860344
test_prec_macro    0.489650   0.843353
test_rec_micro     0.441414   0.699351
train_acc          0.909764   0.989796
train_prec_macro   0.923822   0.966667
train_rec_micro    0.884933   0.996296
                  laplace_smoothing=1.0  laplace_smoothing=10.0  \
fit_time                       0.021799                0.018213   
score_time                    10.089501               11.556496   
test_acc                       0.689676                0.550420   
test_prec_macro                0.489650                0.210388   
test_rec_micro                 0.441414                0.209091   
train_acc                      0.909764                0.559824   
train_prec_macro               0.923822                0.279438   
train_rec_micro                0.884933                0.227249   

                  laplace_smoothing=100.0  
f